## ETL General Servicios de Taxi, Uber y Lyft

## 1. Importar librerías:

In [25]:
import pandas as pd
import seaborn as sns
import warnings
import numpy as np
warnings.filterwarnings('ignore')

## 2. ETL Taxi Amarillo:

### 2.1 Abrir archivo de Taxi Amarillo:

In [26]:
url_Y="https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-12.parquet"

Yellow = pd.read_parquet(url_Y)

Aquí solo se descarga un archivo, pero debe iterarse por cada mes/año en la pagina para descargar los 24 archivos para dos años (2021, 2022 y 2023)

### 2.2 Eliminar comlumnas:

In [27]:
columnas_eliminar = ['VendorID', 'RatecodeID', 'store_and_fwd_flag',
       'payment_type']

Yellow = Yellow.drop(columns=columnas_eliminar)

### 2.3 Eliminar filas con tarifa base menor a $3:

In [28]:
# Eliminar las filas donde 'fare_amount' menor a 3 dólares (tarifa base taxis, $5 la de cancelación uber)
Yellow = Yellow[Yellow['fare_amount'] >= 3]

### 2.4 Imputar valores Errados:

In [29]:
# Calcula la moda de 'congestion_surcharge' y 'Airport_fee' excluyendo los valores cero
moda_congestion = Yellow[Yellow['congestion_surcharge'] != 0]['congestion_surcharge'].mode()[0]
moda_airport = Yellow[Yellow['Airport_fee'] != 0]['Airport_fee'].mode()[0]
moda_Improvement = Yellow[Yellow['improvement_surcharge'] != 0]['improvement_surcharge'].mode()[0]

# Imputa los valores erróneos utilizando la moda
Yellow['congestion_surcharge'] = Yellow['congestion_surcharge'].apply(lambda x: moda_congestion if x > 0 else x)
Yellow['Airport_fee'] = Yellow['Airport_fee'].apply(lambda x: moda_airport if x > 0 else x)
Yellow['Airport_fee'] = Yellow['improvement_surcharge'].apply(lambda x: moda_Improvement if x > 0 else x)


Hay tarifas fijas que se cargan desde el inicio del servicio y otras que dependen si el servicio o dependiendo si val al aeropuerto. Se imputan mediante la moda ya que la mayoría de registros están bien.

### 2.5 Eliminación de Outliers:

In [30]:
# Eliminar las filas donde 'base_passenger_fare' supera los $400
Yellow = Yellow[Yellow['fare_amount'] <= 300] # No se considera real una tarifa mayor $150 y se permite que esta se duplique.

# Eliminar las filas donde 'trip_miles' supera las 100 millas
Yellow = Yellow[Yellow['trip_distance'] <= 100] # la distancia máxima sería 50 millas, se puede suponer ida y vuelta

# Eliminar las filas donde 'tolls' supera los $60 dólares
Yellow = Yellow[Yellow['tolls_amount'] <= 60]

### 2.6 Tranformar fechas y crear tiempos de espera y de viaje:

In [31]:
# 1. Crear la columna "time_out" que sea la diferencia entre tpep_pickup_datetime y request_datetime
Yellow['time_out'] = 0 #En los taxis no hay registro, porque la mayoría son servicion recogidos por parada en la calle

# 2. Crear la columna "travel_time" que sea la diferencia entre dropoff_datetime y tpep_pickup_datetime
Yellow['travel_time'] = Yellow['tpep_dropoff_datetime'] - Yellow['tpep_pickup_datetime']

# 3. Convertir valores negativos en cero
Yellow['travel_time'] = Yellow['travel_time'].clip(lower=pd.Timedelta(0))

# 4. Crear las columnas "year", "month", "day", "hour" a partir de la columna tpep_pickup_datetime
Yellow['year'] = Yellow['tpep_pickup_datetime'].dt.year
Yellow['month'] = Yellow['tpep_pickup_datetime'].dt.month
Yellow['day'] = Yellow['tpep_pickup_datetime'].dt.day
Yellow['hour'] = Yellow['tpep_pickup_datetime'].dt.hour

# 5. Redondear la columna "hour" al entero más cercano (de 1 a 24)
Yellow['hour'] = Yellow['hour'].apply(lambda x: round(x))

# 6. Crear la columna "day_of_week" a partir de la columna tpep_pickup_datetime
Yellow['day_of_week'] = Yellow['tpep_pickup_datetime'].dt.day_name()

# 7. Eliminar las columnas tpep_pickup_datetime y dropoff_datetime
Yellow = Yellow.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])


### 2.7 Imputar tarifas de cargos según horario y unificación  de tarifa base y tarifa extra:

Primero se imputan y corrigen valores de impuestos fijos según horarios:

In [32]:
# Definir una función para asignar los valores según la columna "hour"
def asignar_valor(row):
    if (row['hour'] >= 16 and row['hour'] <= 24) or (row['hour'] <= 6):
        return 1.5 if (row['hour'] >= 16 and row['hour'] <= 20) else 0.5
    else:
        return 0

# Aplicar la función a la columna extra
Yellow['extra'] = Yellow.apply(asignar_valor, axis=1)


Luego se unifican tarifa base y extras:

In [33]:
# Crear la columna "fare_surcharges" que sea la suma de tolls, mta_tax, congestion_surcharge, airport_fee
Yellow['fare_surcharges'] = Yellow['tolls_amount'] + Yellow['extra'] + Yellow['congestion_surcharge'] + Yellow['Airport_fee']

# Crear la columna "base_fare" que sea la suma de base_passenger_fare y sales_tax
Yellow['base_fare'] = Yellow['fare_amount'] + 0.8 # 0.5 MTA + 0.3 Improvement

# Eliminar las columnas de tarifas individuales
columnas_eliminar = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'Airport_fee']
Yellow = Yellow.drop(columns=columnas_eliminar)

### 2.8 Cambiar formato de tiempos de viaje y espera:

In [34]:
# Convertir los datos de timedelta64[us] a minutos y luego a un entero
Yellow['travel_time_minutes'] = Yellow['travel_time'] / pd.Timedelta(minutes=1)

# Convertir a entero
Yellow['travel_time_minutes'] = Yellow['travel_time_minutes'].astype(int)

# Convertir a entero de 32 bits
Yellow['time_out'] = Yellow['time_out'].astype('int32')
Yellow['travel_time'] = Yellow['travel_time_minutes'].astype('int32')

Yellow = Yellow.drop(columns=['travel_time_minutes'])

### 2.9 Eliminar registros donde el tiempo de trayecto sea superior a 240 minutos:

In [35]:
# Eliminar los registros donde el tiempo de trayecto supera las 4 horas (240 minutos)
Yellow =Yellow[Yellow['travel_time'] <= 240]

### 2.10 Imputar fechas mal registradas:

In [36]:
# Servicios que superan 4 horas de trayecto
moda_Año = Yellow[Yellow['year'] != 0]['year'].mode()[0]
moda_Mes = Yellow[Yellow['month'] != 0]['month'].mode()[0]

Yellow['year'] = Yellow['year'].apply(lambda x: moda_Año if x > 0 else x)
Yellow['month'] = Yellow['month'].apply(lambda x: moda_Mes if x > 0 else x)

Aunque no se encontraron en este dataset, en "Green" si y se debe colocar por seguridad. Otra opción es eliminarlos

### 2.11 Crear columnas auxiliares para identificar nuevos outliers Costo por minuto y Costo por milla:

In [37]:
# Filtrar los registros donde trip_distance es cero y asignar un valor NaN
Yellow['$mile'] = np.where(Yellow['trip_distance'] != 0, Yellow['base_fare'] / Yellow['trip_distance'], np.nan)

# Filtrar los registros donde trip_minutes es cero y asignar un valor NaN
Yellow['$minute'] = np.where(Yellow['travel_time'] != 0, Yellow['base_fare'] / Yellow['travel_time'], np.nan)

### 2.11 Eliminar Outliers basado en rango intercuartílico:

In [38]:
# Calcular los cuartiles
Q1_mile = Yellow['$mile'].quantile(0.25)
Q3_mile = Yellow['$mile'].quantile(0.75)
IQR_mile = Q3_mile - Q1_mile

Q1_minute = Yellow['$minute'].quantile(0.25)
Q3_minute = Yellow['$minute'].quantile(0.75)
IQR_minute = Q3_minute - Q1_minute

# Multiplicador para rango intercuartílico:
multiplicador = 2

# Definir los límites superior e inferior
umbral_inf_mile = Q1_mile - multiplicador * IQR_mile
umbral_sup_mile = Q3_mile + multiplicador * IQR_mile

umbral_inf_minute =  Q3_minute - multiplicador * IQR_minute # para evitar borrar todos los registros
umbral_sup_minute = Q3_minute + multiplicador * IQR_minute

# Filtrar los registros que caen dentro del rango intercuartílico para cada columna
Yellow = Yellow[
    (Yellow['$mile'] >= umbral_inf_mile) & (Yellow['$mile'] <= umbral_sup_mile) &
    (Yellow['$minute'] >= umbral_inf_minute) & (Yellow['$minute'] <= umbral_sup_minute)
]

Se usa multiplicador 2 para no eliminar una cantidad significativa de datos. Se puede aumentar para mantener una integridad del promedio.

### 2.13 Eliminar datos con ubicaciones no definidas:

In [41]:
# Eliminar los registros con DOLocationID igual a 264
Yellow = Yellow.loc[Yellow['DOLocationID'] != 264]

# Eliminar los registros con DOLocationID igual a 265
Yellow = Yellow.loc[Yellow['DOLocationID'] != 265]

# Eliminar los registros con PULocationID igual a 264
Yellow = Yellow.loc[Yellow['PULocationID'] != 264]

# Eliminar los registros con PULocationID igual a 265
Yellow = Yellow.loc[Yellow['PULocationID'] != 265]


### 2.14 Imputar registros unificando los polígonos de los ID 55+56 y 103+104+105

In [42]:
# Imputar los valores según las condiciones dadas
Yellow.replace({'PULocationID': {57: 56, 105: 103, 104: 103}, 'DOLocationID': {57: 56, 105: 103,  104: 103}}, inplace=True)

### 2.15 Realizar agregación de Yellow:

In [43]:
# Crear la columna "service_number" y asignar el valor 1 a todas las filas
Yellow['service_number'] = 1

# Crear la columna "service_Type" y asignar el valor Yellow a todas las filas
Yellow['service_type'] = 'Yellow'

# Renombrar columnas (normalizadas a UberLyft) 
Yellow.rename(columns={'trip_distance': 'trip_miles'}, inplace=True)

# Definir las dimensiones de agrupación y las variables de agregación
dimensiones = ['service_type','year', 'month', 'day', 'day_of_week','hour', 'PULocationID', 'DOLocationID']
variables_agregacion = ['trip_miles', 'time_out', 'travel_time', 'fare_surcharges', 'base_fare', 'service_number']

# Agrupar el DataFrame y calcular la suma de las variables de agregación
Yellow = Yellow.groupby(dimensiones)[variables_agregacion].sum().reset_index()

Las operaciones anteririores deben repetirse para todos los archivos (24 o 36 archivos) de taxis amarillos que están por emes en la plataforma de TLC

## 3. ETL Taxis Verdes:

### 3.1 Abrir archivo:

In [68]:
url_G="https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-12.parquet"

Green = pd.read_parquet(url_G)

### 3.2 Eliminar columnas innecesarias:

In [69]:
columnas_eliminar = ['VendorID', 'RatecodeID', 'store_and_fwd_flag',
       'payment_type','trip_type']

Green = Green.drop(columns=columnas_eliminar)

### 3.3 Convertir números negativos a positivos:

In [70]:
# Crear una lista de las columnas en las que deseas convertir los valores negativos a positivos
columnas_a_convertir = ['fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge']

# Aplicar una función lambda a cada valor en cada una de esas columnas para cambiar los valores negativos a positivos
Green[columnas_a_convertir] = Green[columnas_a_convertir].applymap(lambda x: abs(x) if x < 0 else x)

### 3.4 Eliminar filas con tarifas menores a $3:

In [71]:
# Eliminar las filas donde 'fare_amount' menor a 3 dólares (tarifa base taxis, $5 la de cancelación uber)
Green = Green[Green['fare_amount'] >= 3]

### 3.5 Imputar valores errados de recargos fijos o conidicionales al horario:

In [72]:
# Calcula la moda de 'congestion_surcharge' y 'Airport_fee' excluyendo los valores cero
moda_congestion = Green[Green['congestion_surcharge'] != 0]['congestion_surcharge'].mode()[0]
moda_Improvement = Green[Green['improvement_surcharge'] != 0]['improvement_surcharge'].mode()[0]

# Imputa los valores erróneos utilizando la moda
Green['congestion_surcharge'] = Green['congestion_surcharge'].apply(lambda x: moda_congestion if x > 0 else x)
Green['Airport_fee'] = Green['improvement_surcharge'].apply(lambda x: moda_Improvement if x > 0 else x)

Se imputan con la moda dado que la mayoría de los datos están bien al respecto.

### 3.6 Elimnación de Outliers:

In [73]:
# Eliminar las filas donde 'base_passenger_fare' supera los $400
Green = Green[Green['fare_amount'] <= 300] # No se considera real una tarifa mayor $150 y se permite que esta se duplique.

# Eliminar las filas donde 'trip_miles' supera las 100 millas
Green = Green[Green['trip_distance'] <= 100] # la distancia máxima sería 50 millas, se puede suponer ida y vuelta

# Eliminar las filas donde 'tolls' supera los $60 dólares
Green = Green[Green['tolls_amount'] <= 60]

### 3.7 Transformar datos de fecha e incluir tiempos de viaje y espera:

In [75]:
# 1. Crear la columna "time_out" que sea la diferencia entre lpep_pickup_datetime y request_datetime
Green['time_out'] = 0 #En los taxis no hay registro, porque la mayoría son servicion recogidos por parada en la calle

# 2. Crear la columna "travel_time" que sea la diferencia entre dropoff_datetime y lpep_pickup_datetime
Green['travel_time'] = Green['lpep_dropoff_datetime'] - Green['lpep_pickup_datetime']

# 3. Convertir valores negativos en cero
Green['travel_time'] = Green['travel_time'].clip(lower=pd.Timedelta(0))

# 4. Crear las columnas "year", "month", "day", "hour" a partir de la columna lpep_pickup_datetime
Green['year'] = Green['lpep_pickup_datetime'].dt.year
Green['month'] = Green['lpep_pickup_datetime'].dt.month
Green['day'] = Green['lpep_pickup_datetime'].dt.day
Green['hour'] = Green['lpep_pickup_datetime'].dt.hour

# 5. Redondear la columna "hour" al entero más cercano (de 1 a 24)
Green['hour'] = Green['hour'].apply(lambda x: round(x))

# 6. Crear la columna "day_of_week" a partir de la columna lpep_pickup_datetime
Green['day_of_week'] = Green['lpep_pickup_datetime'].dt.day_name()

# 7. Eliminar las columnas lpep_pickup_datetime y dropoff_datetime
Green = Green.drop(columns=['lpep_pickup_datetime', 'lpep_dropoff_datetime'])


### 3.8 Transformar tarifas y unificar tarifas base y tarifas extra:

In [76]:
# Definir una función para asignar los valores según la columna "hour"
def asignar_valor(row):
    if (row['hour'] >= 16 and row['hour'] <= 24) or (row['hour'] <= 6):
        return 1.5 if (row['hour'] >= 16 and row['hour'] <= 20) else 0.5
    else:
        return 0

# Aplicar la función a la columna extra
Green['extra'] = Green.apply(asignar_valor, axis=1)

Unificar tarifas:

In [78]:
# Crear la columna "fare_surcharges" que sea la suma de tolls, mta_tax, congestion_surcharge, airport_fee
Green['fare_surcharges'] = Green['tolls_amount'] + Green['extra'] + Green['congestion_surcharge'] + Green['Airport_fee']

# Crear la columna "base_fare" que sea la suma de base_passenger_fare y sales_tax
Green['base_fare'] = Green['fare_amount'] + 0.8 # 0.5 MTA + 0.3 Improvement

# Eliminar las columnas de tarifas individuales
columnas_eliminar = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge']
Green = Green.drop(columns=columnas_eliminar)

### 3.9 Cambiar formato de tiempos a enteros:

In [79]:
# Convertir los datos de timedelta64[us] a minutos y luego a un entero
Green['travel_time_minutes'] = Green['travel_time'] / pd.Timedelta(minutes=1)

# Convertir a entero
Green['travel_time_minutes'] = Green['travel_time_minutes'].astype(int)

# Convertir a entero de 32 bits
Green['time_out'] = Green['time_out'].astype('int32')
Green['travel_time'] = Green['travel_time_minutes'].astype('int32')

Green = Green.drop(columns=['travel_time_minutes'])

### 3.10 Eliminar registros con tiempo de viaje mayor a 240 minutos:

In [80]:
# Eliminar los registros donde el tiempo de trayecto supera las 4 horas (240 minutos)
Green =Green[Green['travel_time'] <= 240]

### 3.11 Imputar fechas mal registradas de año y mes:

In [82]:
# Servicios que superan 4 horas de trayecto
moda_Año = Green[Green['year'] != 0]['year'].mode()[0]
moda_Mes = Green[Green['month'] != 0]['month'].mode()[0]

Green['year'] = Green['year'].apply(lambda x: moda_Año if x > 0 else x)
Green['month'] = Green['month'].apply(lambda x: moda_Mes if x > 0 else x)

### 3.12 Agregar columnas auxiliares para definir nuevos outliers:

In [83]:
# Filtrar los registros donde trip_distance es cero y asignar un valor NaN
Green['$mile'] = np.where(Green['trip_distance'] != 0, Green['base_fare'] / Green['trip_distance'], np.nan)

# Filtrar los registros donde trip_minutes es cero y asignar un valor NaN
Green['$minute'] = np.where(Green['travel_time'] != 0, Green['base_fare'] / Green['travel_time'], np.nan)

### 3.13 Eliminar outliers mediante rangos intercuartílicos:

In [84]:
# Calcular los cuartiles
Q1_mile = Green['$mile'].quantile(0.25)
Q3_mile = Green['$mile'].quantile(0.75)
IQR_mile = Q3_mile - Q1_mile

Q1_minute = Green['$minute'].quantile(0.25)
Q3_minute = Green['$minute'].quantile(0.75)
IQR_minute = Q3_minute - Q1_minute

# Multiplicador para rango intercuartílico:
multiplicador = 2

# Definir los límites superior e inferior
umbral_inf_mile = Q1_mile - multiplicador * IQR_mile
umbral_sup_mile = Q3_mile + multiplicador * IQR_mile

umbral_inf_minute =  Q3_minute - multiplicador * IQR_minute # para evitar borrar todos los registros
umbral_sup_minute = Q3_minute + multiplicador * IQR_minute

# Filtrar los registros que caen dentro del rango intercuartílico para cada columna
Green = Green[
    (Green['$mile'] >= umbral_inf_mile) & (Green['$mile'] <= umbral_sup_mile) &
    (Green['$minute'] >= umbral_inf_minute) & (Green['$minute'] <= umbral_sup_minute)
]

### 3.14 Eliminar registros con Ubicaciones no definidas:

In [85]:
# Eliminar los registros con DOLocationID igual a 264
Green = Green.loc[Green['DOLocationID'] != 264]

# Eliminar los registros con DOLocationID igual a 265
Green = Green.loc[Green['DOLocationID'] != 265]

# Eliminar los registros con PULocationID igual a 264
Green = Green.loc[Green['PULocationID'] != 264]

# Eliminar los registros con PULocationID igual a 265
Green = Green.loc[Green['PULocationID'] != 265]


### 3.15 Imputar registros unificando los polígonos de los ID 55+56 y 103+104+105:

In [86]:
# Imputar los valores según las condiciones dadas
Green.replace({'PULocationID': {57: 56, 105: 103, 104: 103}, 'DOLocationID': {57: 56, 105: 103,  104: 103}}, inplace=True)

### 3.16 Realizar agregaciones:

In [88]:
# Crear la columna "service_number" y asignar el valor 1 a todas las filas
Green['service_number'] = 1

# Crear la columna "service_Type" y asignar el valor Green a todas las filas
Green['service_type'] = 'Green'

# 
Green.rename(columns={'trip_distance': 'trip_miles'}, inplace=True)


# Definir las dimensiones de agrupación y las variables de agregación
dimensiones = ['service_type','year', 'month', 'day', 'day_of_week','hour', 'PULocationID', 'DOLocationID']
variables_agregacion = ['trip_miles', 'time_out', 'travel_time', 'fare_surcharges', 'base_fare', 'service_number']

# Agrupar el DataFrame y calcular la suma de las variables de agregación
Green = Green.groupby(dimensiones)[variables_agregacion].sum().reset_index()

## 4. ETL UberLyft:

### 4.1 Abrir archivo:

In [107]:
url_UberLyft="https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2023-12.parquet"

UberLyft = pd.read_parquet(url_UberLyft)

### 4.2 Eliminar columnas innecesarias:

In [108]:
columnas_eliminar = ['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'on_scene_datetime','trip_time', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag','tips']

UberLyft = UberLyft.drop(columns=columnas_eliminar)

### 4.3 Convertir números negativos a positivos:

In [109]:
# Crear una lista de las columnas en las que deseas convertir los valores negativos a positivos
columnas_a_convertir = ['base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee']

# Aplicar una función lambda a cada valor en cada una de esas columnas para cambiar los valores negativos a positivos
UberLyft[columnas_a_convertir] = UberLyft[columnas_a_convertir].applymap(lambda x: abs(x) if x < 0 else x)

### 4.4 Eliminar registros con tarifa base menor a $3:

In [110]:
# Eliminar las filas donde 'base_passenger_fare' menor a 3 dólares (tarifa base taxis, $5 la de cancelación uber)
UberLyft = UberLyft[UberLyft['base_passenger_fare'] >= 3]

### 4.5 Imputar valores errados de impuestos fijos o condicionados a hora o lugar:

In [111]:
# Calcula la moda de 'congestion_surcharge' y 'airport_fee' excluyendo los valores cero
moda_congestion = UberLyft[UberLyft['congestion_surcharge'] != 0]['congestion_surcharge'].mode()[0]
moda_airport = UberLyft[UberLyft['airport_fee'] != 0]['airport_fee'].mode()[0]

# Imputa los valores erróneos utilizando la moda
UberLyft['congestion_surcharge'] = UberLyft['congestion_surcharge'].apply(lambda x: moda_congestion if x > 0 else x)
UberLyft['airport_fee'] = UberLyft['airport_fee'].apply(lambda x: moda_airport if x > 0 else x)


Corregir sales_tax al 8.875% de acuerdo con [el siguiente link](https://www.tax.ny.gov/pubs_and_bulls/tg_bulletins/st/translated/sales-tax-rates-spa.htm):

In [112]:
# Filtra los valores no cero en 'sales_tax' y 'base_passenger_fare'
filtered_data = UberLyft[(UberLyft['sales_tax'] != 0) & (UberLyft['base_passenger_fare'] != 0)]

# Calcula la relación entre 'sales_tax' y 'base_passenger_fare' y el promedio redondeado a 2 cifras decimales
promedio_relacion = filtered_data['sales_tax'] / filtered_data['base_passenger_fare']
promedio_redondeado = round(promedio_relacion.mean(), 5)

# Imputar los valores en cero de sales_tax
UberLyft['sales_tax'] = UberLyft['sales_tax']* promedio_redondeado


### 4.6 Eliminar outliers:

In [113]:
# Eliminar las filas donde 'base_passenger_fare' supera los $400
UberLyft = UberLyft[UberLyft['base_passenger_fare'] <= 300] # No se considera real una tarifa mayor $150 y se permite que esta se duplique.

# Eliminar las filas donde 'trip_miles' supera las 100 millas
UberLyft = UberLyft[UberLyft['trip_miles'] <= 100] # la distancia máxima sería 50 millas, se puede suponer ida y vuelta

# Eliminar las filas donde 'tolls' supera los $60 dólares
UberLyft = UberLyft[UberLyft['tolls'] <= 60]

### 4.7 Transformar datos de fecha y adicionar columnas de tiempos de espera y de viaje:

In [114]:
# 1. Crear la columna "time_out" que sea la diferencia entre pickup_datetime y request_datetime
UberLyft['time_out'] = UberLyft['pickup_datetime'] - UberLyft['request_datetime']

# 2. Crear la columna "travel_time" que sea la diferencia entre dropoff_datetime y pickup_datetime
UberLyft['travel_time'] = UberLyft['dropoff_datetime'] - UberLyft['pickup_datetime']

# 3. Convertir valores negativos en cero
UberLyft['travel_time'] = UberLyft['travel_time'].clip(lower=pd.Timedelta(0))
UberLyft['time_out'] = UberLyft['time_out'].clip(lower=pd.Timedelta(0))

# 4. Crear las columnas "year", "month", "day", "hour" a partir de la columna pickup_datetime
UberLyft['year'] = UberLyft['pickup_datetime'].dt.year
UberLyft['month'] = UberLyft['pickup_datetime'].dt.month
UberLyft['day'] = UberLyft['pickup_datetime'].dt.day
UberLyft['hour'] = UberLyft['pickup_datetime'].dt.hour

# 5. Redondear la columna "hour" al entero más cercano (de 1 a 24)
UberLyft['hour'] = UberLyft['hour'].apply(lambda x: round(x))

# 6. Crear la columna "day_of_week" a partir de la columna pickup_datetime
UberLyft['day_of_week'] = UberLyft['pickup_datetime'].dt.day_name()

# 7. Eliminar las columnas request_datetime, pickup_datetime, dropoff_datetime
UberLyft = UberLyft.drop(columns=['request_datetime', 'pickup_datetime', 'dropoff_datetime'])


### 4.8 Transformar datos de tarifas base y recargos:

In [115]:
# Crear la columna "fare_surcharges" que sea la suma de tolls, bcf, congestion_surcharge, airport_fee
UberLyft['fare_surcharges'] = UberLyft['tolls'] + UberLyft['bcf'] + UberLyft['congestion_surcharge'] + UberLyft['airport_fee']

# Crear la columna "base_fare" que sea la suma de base_passenger_fare y sales_tax
UberLyft['base_fare'] = UberLyft['base_passenger_fare'] + UberLyft['sales_tax']

# Eliminar las columnas tolls, bcf, congestion_surcharge, airport_fee, base_passenger_fare y sales_tax
columnas_eliminar = ['tolls', 'bcf', 'congestion_surcharge', 'airport_fee', 'base_passenger_fare', 'sales_tax']
UberLyft = UberLyft.drop(columns=columnas_eliminar)

### 4.9. Cambiar formato de tiempos de viaje y espera:

In [116]:
# Convertir los datos de timedelta64[us] a minutos y luego a un entero
UberLyft['time_out_minutes'] = UberLyft['time_out'] / pd.Timedelta(minutes=1)
UberLyft['travel_time_minutes'] = UberLyft['travel_time'] / pd.Timedelta(minutes=1)

# Convertir a entero
UberLyft['time_out_minutes'] = UberLyft['time_out_minutes'].astype(int)
UberLyft['travel_time_minutes'] = UberLyft['travel_time_minutes'].astype(int)

# Convertir a entero de 32 bits
UberLyft['time_out'] = UberLyft['time_out_minutes'].astype('int32')
UberLyft['travel_time'] = UberLyft['travel_time_minutes'].astype('int32')

UberLyft = UberLyft.drop(columns=['time_out_minutes', 'travel_time_minutes'])


### 4.10 Eliminar outliers de tiempos de viaje y espera:

In [117]:
# Eliminar los registros donde el tiempo de trayecto supera las 4 horas (240 minutos)
UberLyft = UberLyft[UberLyft['travel_time'] <= 240]

# Eliminar los registros donde el tiempo de espera es mayor a 2 horas (120 minutos)
UberLyft = UberLyft[UberLyft['time_out'] <= 120]

# Eliminar los registros donde el tiempo de espera es menor a 1 hora (-60 minutos)
UberLyft = UberLyft[UberLyft['time_out'] >= -60]

### 4.11 Crear columnas auxiliares de costo por minuto y costo por milla para identificar nuevos outliers:

In [119]:
# Filtrar los registros donde trip_miles es cero y asignar un valor NaN
UberLyft['$mile'] = np.where(UberLyft['trip_miles'] != 0, UberLyft['base_fare'] / UberLyft['trip_miles'], np.nan)

# Filtrar los registros donde trip_minutes es cero y asignar un valor NaN
UberLyft['$minute'] = np.where(UberLyft['travel_time'] != 0, UberLyft['base_fare'] / UberLyft['travel_time'], np.nan)


### 4.12. Eliminar los Outliers basado en los rangos intercuartílicos:

Se decide evitar el sesgo de costos que se mantienen muy lejos de la relación Tarifa por milla o Tarifa por minuto así que se eliminan los registros que estén por encima de este cálculo. No se usa 3 sigma debido a que el sesgo de datos atípicos es muy alto. También se decide eliminar registros que superan los 250 dólares por no representar la realidad de un servicio normal dentro de la ciudad de NYC (se puede suponer hasta normal $60, probable $100 y muy poco probable $250 o más).

In [120]:
# Calcular los cuartiles
Q1_mile = UberLyft['$mile'].quantile(0.25)
Q3_mile = UberLyft['$mile'].quantile(0.75)
IQR_mile = Q3_mile - Q1_mile

Q1_minute = UberLyft['$minute'].quantile(0.25)
Q3_minute = UberLyft['$minute'].quantile(0.75)
IQR_minute = Q3_minute - Q1_minute

# Multiplicador para rango intercuartílico:
multiplicador = 2

# Definir los límites superior e inferior
umbral_inf_mile = Q1_mile - multiplicador * IQR_mile
umbral_sup_mile = Q3_mile + multiplicador * IQR_mile

umbral_inf_minute = Q1_minute - multiplicador * IQR_minute
umbral_sup_minute = Q3_minute + multiplicador * IQR_minute

# Filtrar los registros que caen dentro del rango intercuartílico para cada columna
UberLyft = UberLyft[
    (UberLyft['$mile'] >= umbral_inf_mile) & (UberLyft['$mile'] <= umbral_sup_mile) &
    (UberLyft['$minute'] >= umbral_inf_minute) & (UberLyft['$minute'] <= umbral_sup_minute)
]

### 4.13 Eliminar registros de ubicaciones no definidas:

In [121]:
# Eliminar los registros con DOLocationID igual a 264
UberLyft = UberLyft.loc[UberLyft['DOLocationID'] != 264]

# Eliminar los registros con DOLocationID igual a 265
UberLyft = UberLyft.loc[UberLyft['DOLocationID'] != 265]

# Eliminar los registros con PULocationID igual a 264
UberLyft = UberLyft.loc[UberLyft['PULocationID'] != 264]

# Eliminar los registros con PULocationID igual a 265
UberLyft = UberLyft.loc[UberLyft['PULocationID'] != 265]


### 4.14 Imputar registros unificando los polígonos de los ID 55+56 y 103+104+105:

In [122]:
# Imputar los valores según las condiciones dadas
UberLyft.replace({'PULocationID': {57: 56, 105: 103, 104: 103}, 'DOLocationID': {57: 56, 105: 103,  104: 103}}, inplace=True)

### 4.15 Realizar agregacion de UberLyft:

In [124]:
# Crear la columna "service_number" y asignar el valor 1 a todas las filas
UberLyft['service_number'] = 1

# Crear la columna "service_Type" y asignar el valor UberLyft a todas las filas
UberLyft['service_type'] = 'UberLyft'

# Definir las dimensiones de agrupación y las variables de agregación
dimensiones = ['service_type','year', 'month', 'day', 'day_of_week','hour', 'PULocationID', 'DOLocationID']
variables_agregacion = ['trip_miles', 'time_out', 'travel_time', 'fare_surcharges', 'base_fare', 'service_number']

# Agrupar el DataFrame y calcular la suma de las variables de agregación
UberLyft = UberLyft.groupby(dimensiones)[variables_agregacion].sum().reset_index()

## 5. Unir los dataframes:

### 5.1 Unión de datos:

In [129]:
# Unir los dataframes
Servicios_Agrupados = pd.concat([UberLyft, Yellow, Green])

### 5.2 Reducción de tamaño:

In [130]:
columnas_enteras = ['month', 'day', 'hour', 'PULocationID', 'DOLocationID', 'time_out', 'travel_time', 'service_number']
Servicios_Agrupados['year'] = Servicios_Agrupados['year'].astype('int16')

Servicios_Agrupados[columnas_enteras].astype('int8')

# Convertir las columnas float a tipo float32
columnas_float = ['trip_miles', 'fare_surcharges', 'base_fare']
Servicios_Agrupados[columnas_float] = Servicios_Agrupados[columnas_float].astype('float32')

In [ ]:
"""# Eliminar las filas donde 'base_passenger_fare' supera los $400
Servicios_Agrupados = Servicios_Agrupados[Servicios_Agrupados['base_fare'] <= 400] # No se considera real una tarifa mayor $150 y se permite que esta se duplique.

# Eliminar las filas donde 'trip_miles' supera las 100 millas
Servicios_Agrupados = Servicios_Agrupados[Servicios_Agrupados['fare_surcharges'] <= 50] # la distancia máxima sería 50 millas, se puede suponer ida y vuelta"""

In [ ]:
"""# Guardar el DataFrame en un archivo CSV
Servicios_Agrupados.to_parquet('../DataClean/Servicios_Agrupados.parquet', index=False)"""

En este punto se tiene la unión de un mes por los tres tipos de servicio. Esta acción debe repetirse para todos los meses durante los años de interés.